## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [1]:
# !kaggle datasets download -d yutkin/corpus-of-russian-news-articles-from-lenta
# !unzip data/corpus-of-russian-news-articles-from-lenta.zip -d data/

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_path = 'data/'

lenta = pd.read_csv(data_path + 'lenta-ru-news.csv', usecols=['title', 'text', 'topic'])
lenta = lenta[lenta['topic'].notna()]

lenta = lenta.sample(frac=1, random_state=42)

/tmp/ipykernel_39358/2454759729.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta = pd.read_csv(data_path + 'lenta-ru-news.csv', usecols=['title', 'text', 'topic'])


In [4]:
lenta.shape

(738973, 3)

In [5]:
label_dict = {}

for i, topic in enumerate(lenta['topic'].unique()):
    label_dict[topic] = i

label_dict

{'Мир': 0,
 'Наука и техника': 1,
 'Культура': 2,
 'Силовые структуры': 3,
 'Россия': 4,
 'Спорт': 5,
 'Бизнес': 6,
 'Путешествия': 7,
 'Бывший СССР': 8,
 'Дом': 9,
 'Экономика': 10,
 'Интернет и СМИ': 11,
 'Из жизни': 12,
 'Ценности': 13,
 'Культпросвет ': 14,
 '69-я параллель': 15,
 'Крым': 16,
 'Библиотека': 17,
 'Легпром': 18,
 'Оружие': 19,
 'МедНовости': 20,
 'ЧМ-2014': 21,
 'Сочи': 22}

In [6]:
lenta['label'] = lenta['topic'].apply(lambda x: label_dict[x])

lenta['label'].value_counts()

label
4     160445
0     136621
10     79528
5      64413
2      53797
8      53402
1      53136
11     44663
12     27605
9      21734
3      19596
13      7766
6       7399
7       6408
15      1268
16       666
14       340
18       114
17        65
19         3
21         2
20         1
22         1
Name: count, dtype: int64

In [7]:
from nltk import tokenize

tokenizer = tokenize.NLTKWordTokenizer()

In [8]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from string import punctuation

noise = stopwords.words('russian') + list(punctuation)
splitters = ['\'\'', '``', '\"', '-', '\'', '\`']

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [9]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

def morphling_lemmatizer(word):
    parsed_word = morph.parse(word)[0]
    lemma = parsed_word.normal_form
    return lemma

morphling_lemmatizer('деревьев')

'дерево'

In [10]:
from nltk.stem import SnowballStemmer

snowball_stemmer = SnowballStemmer("russian")

def ru_stemmer(word):
    return snowball_stemmer.stem(word)

ru_stemmer('задержание')

'задержан'

In [11]:
def preprocess(sentence): 
    for splitter in splitters:
        sentence = sentence.replace(splitter, ' ')  
    tokens = tokenizer.tokenize(sentence.lower())  
    clean_tokens = [token.strip() for token in tokens if token not in noise]
    lemma_tokens = [morphling_lemmatizer(token) for token in clean_tokens]  
    # stemmed_tokens = [ru_stemmer(token) for token in clean_tokens]
    return ' '.join(lemma_tokens)

preprocess('как открыть карты')

'открыть карта'

In [12]:
# lemma - 77
# stemm - 74

In [13]:
from tqdm import tqdm

title_preprocessed = [preprocess(str(sentence)) for sentence in tqdm(lenta['title'], desc='Preprocessing titles ...')]
text_preprocessed = [preprocess(str(sentence)) for sentence in tqdm(lenta['text'], desc='Preprocessing text ...')]

preprocessed_lenta = pd.DataFrame({
    'title' : title_preprocessed,
    'text' : text_preprocessed,
    'label' : lenta['label']
})

preprocessed_lenta.to_csv(data_path + 'preprocessed_lenta.csv')

Preprocessing text ...: 100%|██████████| 100000/100000 [19:20<00:00, 86.17it/s]


In [20]:
from sklearn.model_selection import train_test_split

def get_test_and_save_train(X_df, y_df, train_path):
    X, y = X_df.tolist(), y_df.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

    with open(data_path + train_path + '.txt', 'w', encoding='utf-8') as file:
        for X_entry, y_entry in zip(X_train, y_train):
            X_entry = str(X_entry).replace('\n', ' ').replace('\r', ' ')
            file.write('__label__' + str(y_entry) + ' ' + X_entry)
            file.write('\n')

    return X_test, y_test
            
X_test, y_test = get_test_and_save_train(preprocessed_lenta['text'], preprocessed_lenta['label'], train_path='lenta_train')

In [21]:
# ! git clone https://github.com/facebookresearch/fastText.git
# ! pip3 install fastText/.

In [22]:
import fasttext

ft_model = fasttext.train_supervised(
    input=data_path+'lenta_train.txt',
    label='__label__',
    lr=0.5,
    epoch=25,
    wordNgrams=2, 
    dim=200,
    thread=2,
    verbose=3000
)

Read 10M words
Number of words:  282995
Number of labels: 19
Progress: 100.0% words/sec/thread: 1122841 lr:  0.000000 avg.loss:  0.101565 ETA:   0h 0m 0s 57.3% words/sec/thread: 1138867 lr:  0.213359 avg.loss:  0.163717 ETA:   0h 0m50s 74.0% words/sec/thread: 1138807 lr:  0.129787 avg.loss:  0.131436 ETA:   0h 0m30s 88.6% words/sec/thread: 1126190 lr:  0.057134 avg.loss:  0.112834 ETA:   0h 0m13s


In [23]:
label = ft_model.predict('армия')[0]
label

('__label__1',)

In [24]:
predicted_labels = ft_model.predict(X_test)[0]
predicted_labels = [int(label[0][9:]) for label in predicted_labels]

In [25]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("Accuracy score: ", accuracy_score(y_test, predicted_labels))
print("Precision score: ", precision_score(y_test, predicted_labels, average='weighted'))
print("Recall score: ", recall_score(y_test, predicted_labels, average='weighted'))
print("f1-score: ", f1_score(y_test, predicted_labels, average='weighted'))

report = classification_report(y_test, predicted_labels)
print("\nClassification Report:\n", report)

Accuracy score:  0.8232
Precision score:  0.8210461749732852
Recall score:  0.8232
f1-score:  0.8211251563795954

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.84      0.82      4521
           1       0.84      0.85      0.84      1877
           2       0.87      0.89      0.88      1792
           3       0.72      0.59      0.65       671
           4       0.80      0.84      0.82      5471
           5       0.97      0.96      0.96      2187
           6       0.64      0.49      0.56       279
           7       0.78      0.69      0.73       211
           8       0.85      0.80      0.82      1787
           9       0.87      0.83      0.85       695
          10       0.84      0.86      0.85      2692
          11       0.77      0.73      0.75      1541
          12       0.66      0.59      0.62       926
          13       0.89      0.81      0.85       265
          14       0.00      0.00      0.00        